In [5]:
# Restart kernel and reimport everything to ensure fresh imports
import importlib
import sys

# Clear any cached modules
if 'grand_file' in sys.modules:
    del sys.modules['grand_file']
if 'geolib' in sys.modules:
    del sys.modules['geolib']

import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json
sys.path.append('../Setup')  # Go up one level to reach Setup folder
from gee_setup import *

setup_gee()

# Import with fresh reload
from grand_file import snow_difference_map

✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
Google Earth Engine initialized successfully!
EE version: 1.6.3


In [4]:
ee.Authenticate()


Successfully saved authorization token.


In [ ]:
## define aoi polygon
aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])

snow_difference_map(aoi, output_filename='25_cloudcover_map', cloud_cover=25, output_format='png')

Found 19 images for period 1 and 11 images for period 2
Found 23 images for period 1 and 20 images for period 2
Found 23 images for period 1 and 20 images for period 2


In [ ]:
# Test PNG export with Rocky Mountains
snow_difference_map(aoi, 
                   output_filename='rocky_mountains_test', 
                   output_format='png',  # Export as PNG instead of HTML
                   cloud_cover=25)

# Also test TIFF export for high quality
snow_difference_map(aoi, 
                   output_filename='rocky_mountains_high_quality', 
                   output_format='tiff',  # High quality TIFF export
                   cloud_cover=25)

In [10]:
# Define coordinates for a region in Peru/Bolivia Andes
aoi2_coords = [[-73.69599934532533,-12.244453922112815],
               [-74.17939778282533,-14.998901983199628],
               [-68.70820637657533,-16.731947710273676],
               [-67.76338215782533,-13.636366870416724],
               [-73.69599934532533,-12.244453922112815]]

# Convert to Earth Engine Geometry
aoi2 = ee.Geometry.Polygon([aoi2_coords])

# Run analysis with appropriate parameters for Andes region
snow_difference_map(aoi2, 
                   output_filename='andes_snow_difference_map_june', 
                   output_format='html', 
                   month_int=6,  # October for Southern Hemisphere
                   cloud_cover=30)  # Higher cloud cover for mountainous region